In [ ]:
import csv
import pandas as pd
import geopandas as gpd
from math import atan, pi
import numpy as np
import shapely
import folium
import warnings
import sys
import torch
import pickle
import copy
import pickle
from itertools import permutations,combinations
import random
import math
from shapely.geometry import Polygon
from geopy.distance import geodesic
from shapely.ops import unary_union
from collections import defaultdict
order_df=pd.read_csv('order_df_pred.csv')
real_df=pd.read_csv('order_df_real.csv')
date_list = order_df.columns[4:382].tolist()
poi_info=pd.read_csv('aoi_poi_infos.csv')
gt=np.load('attendance.npy')
lq_key=pd.read_csv('region_keys.csv')
data=torch.load('sample_last.pt')
with open('distances.pkl', 'rb') as f:
    distances = pickle.load(f)
# aoi_dict = AOI.get_aoi_bound_online(site_id_list=[])
# lq_dict=AOI.get_road_bound_online(site_id=)

In [1]:
def get_neighbor_pair(order_df):
    large_area_dict=lq_dict
    small_area_dict=aoi_dict
    aoi_to_large_dict = {row['aoi_id']: row['路区'] for _, row in order_df.iterrows()}
    adjacent_aoi_pairs = set()
    for aoi_id1, aoi_polygon1 in aoi_dict.items():
        for aoi_id2, aoi_polygon2 in aoi_dict.items():            
            if aoi_polygon1.touches(aoi_polygon2) and aoi_to_large_dict[aoi_id1] != aoi_to_large_dict[aoi_id2]:
                adjacent_aoi_pairs.add((aoi_id1, aoi_id2, aoi_to_large_dict[aoi_id2]))
    d = {}
    for a, b, c in adjacent_aoi_pairs:
        key = (a, c)
        if key not in d:
            d[key] = [(a, c)]
        else:
            d[key].append((a, c))
    new_s = set()
    for v in d.values():
        new_s.add(v[0])
    pairs=pd.DataFrame(new_s,columns=['aoi','lq'])
    pairs['w']=pairs.apply(lambda x: order_df.loc[order_df['aoi_id']==x['aoi'],'Avg'].unique()[0],axis=1)
    st=set(tuple(row) for row in pairs.values)
    return st

In [ ]:
def generate_orders_dataframe(date, df):
    result = pd.DataFrame(columns=['订单序号', 'dt', 'coarse_aoi_id', 'operator_user'])
    formatted_date = pd.to_datetime(date, format='%Y%m%d').strftime('%Y-%m-%d')
    for index, row in df.iterrows():
        order_count = row[date]
        temp_df = pd.DataFrame({
            '订单序号': range(1, int(order_count) + 1),
            'dt': formatted_date,
            'coarse_aoi_id': row['aoi_id'],
            'operator_user': row['路区']
        })
        result = pd.concat([result, temp_df])
    result.reset_index(drop=True, inplace=True)
    return result

In [ ]:
def are_adjacent(polygon1, polygon2):
    return polygon1.boundary.intersects(polygon2.boundary)
adjacency_matrix = np.zeros((len(lq_dict), len(lq_dict)))
for i, polygon1 in enumerate(lq_dict.values()):
    for j, polygon2 in enumerate(lq_dict.values()):
        if are_adjacent(polygon1, polygon2):
            adjacency_matrix[i, j] = 1
adjacency_df = pd.DataFrame(adjacency_matrix, index=lq_dict.keys(), columns=lq_dict.keys())

hops_df = adjacency_df.copy()
for i in lq_dict.keys():
        for j in lq_dict.keys():
            if i!=j and hops_df.loc[i, j]==0:
                hops_df.loc[i, j] = 100
for k in lq_dict.keys():
    for i in lq_dict.keys():
        for j in lq_dict.keys():
            hops_df.loc[i, j] = min(hops_df.loc[i, j], hops_df.loc[i, k] + hops_df.loc[k, j])
# hops_df refers to the matrix that indicate the hops between the aois

In [ ]:
def get_min_distance(courier, target_aoi, df, distance, vac_courier):
    if courier not in vac_courier:
        courier_aois = df[df['operator_user'] == courier]['coarse_aoi_id'].unique()
        # calculate the distances of the target aoi to all other aois
        distances = [distance[(aoi, target_aoi)] for aoi in courier_aois]
        return min(distances)
    else:
        return 1000000;
def get_time(df, date):
    # make sure "dt" is datetime format 
    df['dt'] = pd.to_datetime(df['dt'])    
    one_month_ago = pd.to_datetime(date) - pd.DateOffset(months=1)
    df = df[(df['dt'] >= one_month_ago) & (df['dt'] < date)]
    df_time = get_regression_time(df,date,poi_info).to_dict()
    return df_time

def simulated_annealing_tsp(locations, distances, temp=1000, alpha=0.995, max_iter=500):
    current_path = locations[:]
    best_path = locations[:]
    final_path=locations[:]
    random.shuffle(current_path)
    def path_distance(path):
        return sum(distances[(path[i-1], path[i])] for i in range(len(path)-1))
    current_distance = path_distance(current_path)
    final_distance = current_distance
    for j in range(10):
        random.shuffle(current_path)
        current_distance = path_distance(current_path)
        best_distance = current_distance
        for i in range(max_iter):
            temp *= alpha  # temp decay
            # random switch
            city1, city2 = random.sample(range(len(locations)), 2)
            new_path = current_path[:]
            new_path[city1], new_path[city2] = new_path[city2], new_path[city1]
            new_distance = path_distance(new_path)
            if new_distance < current_distance or random.random() < math.exp((current_distance - new_distance) / temp):
                current_path = new_path
                current_distance = new_distance
                if new_distance < best_distance:
                    best_distance = current_distance
                    best_path = current_path[:]
        if best_distance<final_distance:
            final_distance=best_distance
            final_path=best_path[:]
    return final_distance

def calc_time(df_time,df,vac_aoi=[]):
    dist_dict={}
    time_dict={}
    for operator in df['operator_user'].unique():
        locations = df.loc[df['operator_user']==operator,'coarse_aoi_id'].unique()
        if len(locations)>=2:            
            dist_dict[operator]=simulated_annealing_tsp(locations,distances)
        else:
            dist_dict[operator]=0
        time_dict[operator]=dist_dict[operator]/1000*15 # where 15 refers to 15km/h, the average speed of the e-bicycle
    for aoi in df['coarse_aoi_id'].unique():
        operator=df.loc[df['coarse_aoi_id']==aoi,'operator_user'].unique()[0]
        time_dict[operator]+=len(df[df['coarse_aoi_id']==aoi])*df_time[aoi]
        return time_dict
def order_prior(absent_courier,df,courier_df,time_dict,distances):
    absent_courier=absent_courier.copy()
    test_od_df1=df.copy()
    df_time=order_df.set_index('aoi_id')['time'].to_dict()
    courier_order_counts = test_od_df1['operator_user'].value_counts()
    aoi_order_counts = test_od_df1['coarse_aoi_id'].value_counts()
    absent_courier_aois = test_od_df1[test_od_df1['operator_user'].isin(absent_courier)]['coarse_aoi_id'].unique()
    absent_courier_aoi_order_counts = test_od_df1[test_od_df1['coarse_aoi_id'].isin(absent_courier_aois)]['coarse_aoi_id'].value_counts()
    sorted_absent_courier_aoi_order_counts = absent_courier_aoi_order_counts.sort_values(ascending=False)
    for aoi, aoi_count in sorted_absent_courier_aoi_order_counts.iteritems():
        absent_lq=test_od_df1.loc[test_od_df1['coarse_aoi_id']==aoi,'operator_user'].unique()[0]
        sorted_courier_order_counts = courier_order_counts.sort_values()        
        least_busy_courier = sorted_courier_order_counts.idxmin()
        while hops_df.loc[least_busy_courier,absent_lq]>=3:
            sorted_courier_order_counts = sorted_courier_order_counts.drop(least_busy_courier)
            least_busy_courier = sorted_courier_order_counts.idxmin()
        # update order volume for each courier
        sorted_courier_order_counts[least_busy_courier] += aoi_count
        test_od_df1.loc[test_od_df1['coarse_aoi_id']==aoi,'operator_user']=least_busy_courier
    df_time=order_df.set_index('aoi_id')['time'].to_dict()
    courier_order_counts = test_od_df1['operator_user'].value_counts()
    order_avg=courier_order_counts.mean()
    order_var=courier_order_counts.var()
    time_dict=calc_time(df_time,test_od_df1)
    time_avg=np.mean(list(time_dict.values()))
    time_var=np.var(list(time_dict.values()))
    return order_avg,order_var,time_avg,time_var,list(time_dict.values())       
def dist_prior(absent_courier,df,courier_df,time_dict,distances):
    test_od_df1=df.copy()
    absent_courier=absent_courier.copy()
    df_time=order_df.set_index('aoi_id')['time'].to_dict()
    courier_order_counts = test_od_df1['operator_user'].value_counts()
    time_dict=calc_time(df_time,test_od_df1)
    time_avg=np.mean(list(time_dict.values()))
    time_var=np.var(list(time_dict.values()))
    ori=(courier_order_counts.mean(),courier_order_counts.var(),time_avg,time_var)
    aoi_order_counts = test_od_df1['coarse_aoi_id'].value_counts()
    absent_courier_aois = test_od_df1[test_od_df1['operator_user'].isin(absent_courier)]['coarse_aoi_id'].unique()
    absent_courier_aoi_order_counts = test_od_df1[test_od_df1['coarse_aoi_id'].isin(absent_courier_aois)]['coarse_aoi_id'].value_counts()
    sorted_courier_order_counts = courier_order_counts.sort_values()
    sorted_absent_courier_aoi_order_counts = absent_courier_aoi_order_counts.sort_values(ascending=False)
    for aoi in absent_courier_aois:
        cur_courier_df=courier_df[~np.isin(courier_df,absent_courier)]        
        courier_distances = {courier: get_min_distance(courier, aoi, test_od_df1, distances,absent_courier) for courier in cur_courier_df}
        df_time=order_df.set_index('aoi_id')['time'].to_dict()
        min_distance_courier = min(courier_distances, key=courier_distances.get)
        bk_up_courier=copy.copy(min_distance_courier)
        while calc_courier_time(test_od_df1,df_time,min_distance_courier,distances)>300:
            if len(courier_distances)>=2:
                absent_courier.append(min_distance_courier)
                cur_courier_df=courier_df[~np.isin(courier_df,absent_courier)]
                courier_distances = {courier: get_min_distance(courier, aoi, test_od_df1, distances,absent_courier) for courier in cur_courier_df}
                min_distance_courier = min(courier_distances, key=courier_distances.get)
            else:
                min_distance_courier=bk_up_courier
                break
        test_od_df1.loc[test_od_df1['coarse_aoi_id'] == aoi, 'operator_user'] = min_distance_courier
    df_time=order_df.set_index('aoi_id')['time'].to_dict()
    courier_order_counts = test_od_df1['operator_user'].value_counts()
    order_avg=courier_order_counts.mean()
    order_var=courier_order_counts.var()
    time_dict=calc_time(df_time,test_od_df1)
    time_avg=np.mean(list(time_dict.values()))
    time_var=np.var(list(time_dict.values()))
    return order_avg,order_var,time_avg,time_var,list(time_dict.values())

In [ ]:
def calc_metric(order_df,sample=100,date_num=30):
    cur_order_df=order_df.copy()
    ares=[]
    stm=time.time()
    num=range(100)
    rand_sample=random.sample(num,sample)
    for i in rand_sample:
        for date in range(date_num):
            test_od_df1=generate_orders_dataframe(date_list[348+date],cur_order_df)
            bool_lst = [tensor.item() != 0 for tensor in list(data[i,:,date])]
            absent=[ele for i,ele in enumerate(lq_key) if not bool_lst[i]]
            avg_time=order_df.set_index('aoi_id')['time'].to_dict()
            time_dict=calc_time(avg_time,test_od_df1)
            courier_df=order_df['路区'].unique()
            result2=order_prior(absent,test_od_df1,courier_df,time_dict,distances)
            ares.append([i,date_list[348+date],result2])
    metric=[max(item[2][2]-480,0) for item in ares]
    return metric
def calc_real_metric_order(order_df,sample=1,date_num=30,count=1,bench=480):
    cur_order_df=order_df.copy()
    ares=[]
    metric=[0 for i in range(count)]
    stm=time.time()
    for j in range(sample):
        for date in range(date_num):
            test_od_df1=generate_orders_dataframe(date_list[348+date],cur_order_df)
            bool_lst = [a != 0 for a in gt[:,date+348]]
            absent=[ele for i,ele in enumerate(lq_key) if not bool_lst[i]]
            avg_time=order_df.set_index('aoi_id')['time'].to_dict()
            time_dict=calc_time(avg_time,test_od_df1)
            courier_df=order_df['路区'].unique()
            result2=order_prior(absent,test_od_df1,courier_df,time_dict,distances)
            ares.append([j,date_list[348+date],result2])
    mid_res=[item[2][4] for item in ares]
    for i in range(count):
        metric[i]=np.mean([np.mean(np.maximum(np.array(lst)-(int(bench)-30*int(i)),0)) for lst in mid_res])
    return metric
def calc_real_metric_dist(order_df,sample=1,date_num=30,count=1,bench=480):
    cur_order_df=order_df.copy()
    ares=[]
    metric=[0 for i in range(count)]
    stm=time.time()
    for j in range(sample):
        for date in range(date_num):
            test_od_df1=generate_orders_dataframe(date_list[348+date],cur_order_df)
            bool_lst = [a != 0 for a in gt[:,date+348]]
            absent=[ele for i,ele in enumerate(lq_key) if not bool_lst[i]]
            avg_time=order_df.set_index('aoi_id')['time'].to_dict()
            time_dict=calc_time(avg_time,test_od_df1)
            courier_df=order_df['路区'].unique()
            result1=dist_prior(absent,test_od_df1,courier_df,time_dict,distances)
            ares.append([j,date_list[348+date],result1])
    mid_res=[item[2][4] for item in ares]
    for i in range(count):
        metric[i]=np.mean([np.mean(np.maximum(np.array(lst)-(int(bench)-30*int(i)),0)) for lst in mid_res])
    return metric

In [ ]:
max_iter=100
best_metric=100000
best_res=copy.copy(order_df)
acpt_count=0
valid_df_list=[]
accept_time=0
for it in range(max_iter):
    cur_order_df=copy.copy(best_res)
    print('current iteration：',it)    
    print('searching possible aoi pairs...')
    adjacent_small_area_pairs=get_neighbor_pair(cur_order_df)
    target=random.sample(list(adjacent_small_area_pairs),1)
    print(target)
    ori_lq=cur_order_df.loc[cur_order_df['aoi_id']==target[0][0],'路区'].unique()[0]    
    cur_order_df.loc[cur_order_df['aoi_id']==target[0][0],'路区']=target[0][1]
    ori_lq_aoi=[aoi_dict[aoi] for aoi in cur_order_df.loc[cur_order_df['路区']==ori_lq,'aoi_id'].unique()]
    union_result =unary_union(ori_lq_aoi)
    if union_result.geom_type != 'MultiPolygon':
        cur_metric=calc_metric(cur_order_df,sample=10,date_num=30)
        if get_vote_res(cur_metric,best_metric)>5:
            best_metric=cur_metric
            best_res=copy.copy(cur_order_df)
            accept_time+=1
            print('Yes! The modification is accepted, current metric:',best_metric)    
        else:
            print('No! The modification is NOT accepted, best metric:',best_metric,'current metric:',cur_metric)